<a href="https://colab.research.google.com/github/dhallankit/INFM737_SalesforceReportETL/blob/master/INFM737_Capstone.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Salesforce Report ETL Process

Select Runtime from the menu and click Run all

In [0]:
#Importing Essential Libraries

import pandas as pd
import io
import pdb
from datetime import datetime

## Choose file to be uploaded

Please name the file as "Activity Log with Companies.csv"

In [0]:
from google.colab import files
uploaded = files.upload()

Saving Activity Log with Companies.csv to Activity Log with Companies.csv


In [0]:
#Loading the file into a dataframe
impact_record = pd.read_csv(io.BytesIO(uploaded['Activity Log with Companies.csv']), encoding= 'unicode_escape')

In [0]:
#top 10 rows of the report
impact_record.head()

,Subject,Account ID,Account Classification,DB Activity Type,Date,Account Name,Activity Type
0,Email: MAVRIC 2020,0013900001mtbyd,NGO,Email,1/15/2020,AARP,Sponsorship Prospect
1,Meeting,0013p00001oqRW6,Startup,Meeting,2/6/2020,ABSI Defense,Follow-up Meeting; Sponsorship Prospect; Techn...
2,Meeting,0013900001mtbyg,Corp,Meeting,1/21/2020,Accenture,Follow-up Meeting; Joint Research Project Support
3,Meeting,0013900001mtbyg,Corp,Meeting,10/24/2019,Accenture,Initial Meeting
4,Meeting,0013900001mtbyg,Corp,Meeting,11/1/2019,Accenture,Initial Meeting; Joint Research Project Suppor...


In [0]:
activity_count  = {}
for i, row in impact_record.iterrows():
  if row['Account Name'] in activity_count:
    activity_count[row['Account Name']] +=1
  else:
    activity_count[row['Account Name']] = 1

In [0]:
impact_record['Activity Type'] = impact_record['Activity Type'].str.split('; ')
s = impact_record.apply(lambda x: pd.Series(x['Activity Type']), axis=1).stack().reset_index(level=1, drop=True)
s.name = 'Activity Type'
impact_record = impact_record.drop('Activity Type', axis = 1).join(s).reset_index(drop=True)

In [0]:
common_field = ['Account ID', 'Account Classification', 'Date', 'Account Name', 'DB Activity Type']

column_names = ["Account Name", "Account ID", "Account Classification", "DB Activity Type", "Date", "# of Technical Assistance Meetings",
               "# of Mentoring & Coaching Sessions", "# of Meetings with Potential Investors", "# of SBIR Award Applications Supported",
               "# of Joint Research Projects", "Business Assistance", "Job Search", "# of Patent Applications in Process", "# of Patents Held", "# of Employees with Increased Capacity",
               "# of Products Launched", "# of Concepts Reviewed / Supported", "Funding Decks/Applications Reviewed", "Outreach Events/ Partnership",
               "Amt of Capital Raised By Loans & Grants", "Amt of Investment Capital Raised", "Past Fiscal Year Business Sales", "Total Business Sales",
               "# of Revenue Opportunities Presented", "# of Speakers", "Total Number of Activities", "Tracking Date"]

df = pd.DataFrame(columns = column_names)

In [0]:
activity_dict = {'Application Support':'Funding Decks/Applications Reviewed',
'Case Request Support':'# of Concepts Reviewed / Supported',
'Funding Decks / Application Review':'Funding Decks/Applications Reviewed',
'Joint Research Project Support':'# of Joint Research Projects',
'Legal Advice':'Business Assistance',
'Marketing':'Business Assistance',
'Mentoring & Coaching':'# of Mentoring & Coaching Sessions',
'Potential Investors':'# of Meetings with Potential Investors',
'SBIR':'# of SBIR Award Applications Supported',
'Technical Assistance':'# of Technical Assistance Meetings',
'Conference Organization':'Outreach Events/ Partnership',
'Connection Request':'Business Assistance',
'Employment Opp':'# of Employees with Increased Capacity',
'Market Advice':'Business Assistance',
'Outreach Event':'Outreach Events/ Partnership',
'Partnership Opportunity':'Outreach Events/ Partnership',
'Patent Application':'# of Patent Applications in Process',
'Products Launched':'# of Products Launched',
'Revenue Opp':'# of Revenue Opportunities Presented',
'Seeking Employment':'Job Search',
'Sponsorship Prospect':'Outreach Events/ Partnership',
'Vendor Search':'Business Assistance'}
activity_field = list(activity_dict.values())

In [0]:
no_mapping_field = ["# of Patents Held", "# of Employees with Increased Capacity", "Amt of Capital Raised By Loans & Grants", "Amt of Investment Capital Raised", "Past Fiscal Year Business Sales", "Total Business Sales",
                "# of Speakers", "Total Number of Activities"] 

activity_df = pd.DataFrame(columns=activity_field+no_mapping_field)

account_name_unique_list = impact_record['Account Name'].unique()
for account_name in account_name_unique_list:
    filt_record = impact_record[impact_record['Account Name'] == account_name]
    filt_record.sort_values(by = 'Date', ascending=False)
    # pdb.set_trace()
    df = pd.concat([df, filt_record[common_field].head(1)])

df[activity_field+no_mapping_field]=0


In [0]:
for account_name in account_name_unique_list:
    for akey,aval in activity_dict.items():
        mycnt = len(impact_record[(impact_record['Account Name'] == account_name) & (impact_record['Activity Type'] == akey)])
        # pdb.set_trace()
        df.loc[df['Account Name'] == account_name, aval] += mycnt

In [0]:
track_series = []
for date_string in df['Date']:
    date_obj = datetime.strptime(date_string, '%m/%d/%Y')
    if date_obj.month >= 10:
        tr_year = date_obj.year+1
    else:
        tr_year = date_obj.year
    if date_obj.month >= 4 and date_obj.month <= 9:
        tr_month = 9
        tr_day = 30
    else:
        tr_month = 3
        tr_day = 31
    track_series.append(f"{tr_month}/{tr_day}/{tr_year}")
        
        
df['Tracking Date'] = track_series
df.replace(0, '', inplace=True)

In [0]:
for i, row in df.iterrows():
  row['Total Number of Activities'] = activity_count[row['Account Name']]

In [0]:
#This block exports the file with all companies no matter if they had activities or not
latest_tracking_date = df['Tracking Date'][1] 
file_date = datetime.strptime(latest_tracking_date, '%m/%d/%Y')
file_name = (f"Impact Report {file_date.month}-{file_date.day}-{file_date.year}.csv")

df.to_csv(file_name, index=None, header = True)
files.download(file_name)


In [0]:
#This block only exports companies with activities
df2 = df.replace(to_replace = '', value=pd.np.nan)
df2.dropna(subset=['# of Technical Assistance Meetings',
       '# of Mentoring & Coaching Sessions',
       '# of Meetings with Potential Investors',
       '# of SBIR Award Applications Supported',
       '# of Joint Research Projects', 'Business Assistance', 'Job Search',
       '# of Patent Applications in Process', '# of Patents Held',
       '# of Employees with Increased Capacity', '# of Products Launched',
       '# of Concepts Reviewed / Supported',
       'Funding Decks/Applications Reviewed', 'Outreach Events/ Partnership',
       'Amt of Capital Raised By Loans & Grants',
       'Amt of Investment Capital Raised', 'Past Fiscal Year Business Sales',
       'Total Business Sales', '# of Revenue Opportunities Presented',
       '# of Speakers'], how= 'all', inplace=True)
df2.to_csv(str(file_name+'_Only Companies with Activities'), index=None, header = True)
files.download(str(file_name+'_Only Companies with Activities'))

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:1: FutureWarning: The pandas.np module is deprecated and will be removed from pandas in a future version. Import numpy directly instead
  """Entry point for launching an IPython kernel.
